In [5]:
import imageio

img_arr = imageio.v2.imread("./dog1.jpg")
img_arr.shape

(1280, 1045, 3)

In [12]:
import torch

img_t = torch.from_numpy(img_arr)
img_t = img_t.permute(2,0,1) #change dim positions
img_t.shape

torch.Size([3, 1280, 1045])

In [42]:
import os

data_dir = "./image-cats/"
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == ".png"]

batch_size = 3
n_channels = 3
batch = torch.zeros(batch_size, n_channels, 256, 256, dtype = torch.uint8)
# print(batch[0])

for i, name in enumerate(filenames):
    img_arr = imageio.v2.imread(os.path.join(data_dir, name))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    batch[i] = img_t

batch = batch.float()
batch = batch / 255.0
# print(batch[0])

for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:,c] = (batch[:,c] - mean) / std

# print(batch[0])

In [51]:
data_dir = "./volumetric-dicom/2-LUNG 3.0  B70f-04083/"

vol_arr = imageio.volread(data_dir, "DICOM")
vol_arr.shape

vol_t = torch.from_numpy(vol_arr)
batch = vol_t.unsqueeze(0).unsqueeze(0) # add dimentions

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


In [88]:
import numpy as np

data_path = "./tabular-wine/winequality-white.csv"
table_arr = np.loadtxt(data_path, dtype=np.float32, delimiter=';', skiprows=1)
table_t = torch.from_numpy(table_arr)

data_t = table_t[:,:-1] #features
target_t = table_t[:,-1] #lables

#1 : (1,0,0,0,0,0,0,0,0,0)
#2 : (0,1,0,0,0,0,0,0,0,0) ...
one_hot_target_t = torch.zeros(target_t.shape[0], 10).long()
one_hot_target_t.scatter_(1, target_t.unsqueeze(1).long(), 1.0)

#normaliztion:
data_mean = torch.mean(data_t, dim=0)
data_std = torch.std(data_t, dim=0)
data_norm = (data_t - mean)/data_std

# (target_t <= 3).sum()

bad = data_t[(target_t <= 3)]
medium = data_t[(target_t >= 3) & (target_t < 7)]
good = data_t[(target_t >= 7)]

bad.shape, medium.shape, good.shape

(torch.Size([20, 11]), torch.Size([3838, 11]), torch.Size([1060, 11]))